In [ ]:
# Evolutionary Computing Project Report

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import json

# Load the data
with open('experiment_results.json') as f:
    data = json.load(f)

# Function to extract metrics
def extract_metrics(data, experiment_name):
    iterations = range(len(data[experiment_name]))
    mean_fitness = [np.mean(iteration["fitness"]) for iteration in data[experiment_name]]
    max_fitness = [np.max(iteration["fitness"]) for iteration in data[experiment_name]]
    mean_links = [np.mean(iteration["links"]) for iteration in data[experiment_name]]
    return iterations, mean_fitness, max_fitness, mean_links

# Extract metrics for control experiment
control_iterations, control_mean_fitness, control_max_fitness, control_mean_links = extract_metrics(data, 'control_experiment')

# Extract metrics for encoding scheme experiments
# Replace 'encoding_scheme_experiment_name' with actual experiment names
encoding_experiments = ['evolve_motor_controls', 'evolve_link_length_and_motor_controls', 'high_population_high_mutation', 'low_population_high_crossover']
encoding_metrics = {exp: extract_metrics(data, exp) for exp in encoding_experiments}


<!-- # Evolutionary Computing Project Report {-} -->

*Done by: Carlo Imatong*

## Introduction {-}
Evolutionary computing is a fascinating branch of artificial intelligence that leverages mechanisms inspired by biological evolution, such as reproduction, mutation, recombination, and selection. This project focuses on creating and evolving creatures within a simulated environment using genetic algorithms. The primary objective is to explore how different encoding schemes and experimental setups impact the evolution of these creatures in terms of fitness and complexity.

## Basic Experiments {-}
The initial experiments aimed to establish a baseline for the creatures’ evolution using a control setup. These experiments utilized a simple genetic algorithm with a defined population size, gene count, crossover rate, and mutation rates. The primary goal was to observe how these parameters affect the fitness and structure of the evolved creatures over multiple iterations.

### Experiment Parameters {-}
| Experiment Name    | Population Size | Gene Count | Crossover Rate | Point Mutation Rate | Shrink Mutation Rate | Grow Mutation Rate | Non-Evolvable Genes |
|--------------------|-----------------|------------|----------------|---------------------|----------------------|--------------------|---------------------|
| Control Experiment | 20              | 3          | 0.9            | 0.1                 | 0.25                 | 0.1                | None                |

These parameters were carefully selected to provide a balanced approach to evolution, allowing for sufficient genetic diversity while maintaining a manageable population size for computational feasibility.

## Fitness Function {-}
The fitness function os a cruicial component of the evolutionary algorithm as it determines the quality of each creature. In this project, the fitness function evaluates creatures based on three key components: 

- Distance Traveled
- Proximity to Mountain
- Height Achieved

### Calculation of Fitness {-}
The fitness function is calculated as follows:
${ \text{Fitness} = (w_{\text{distance}} \times \text{Distance Traveled}) + (w_{\text{proximity}} \times \frac{1}{\text{Proximity to Mountain} + 1}) + (w_{\text{height}} \times \text{Height Achieved}) }$

- **Distance Traveled:** This component measures how far the creature has moved from its starting position. It encourages exploration and mobility.
- **Proximity to Mountain:** This component rewards creatures that get closer to the mountain, which simulates the ability to navigate towards a goal.
- **Height Achieved:** This component rewards creatures that achieve higher altitudes whilst touching the mountain, simulating the ability of climbing the mountain, and encourages the evolution of creatures capable of climbing.

### Weight factors {-}

The weights for each component are chosen to balance their contributions to the overall fitness:

- **Weight for Distance Traveled ( $w_{\text{distance}}$ ): 0.1**
- **Weight for Proximity ( $w_{\text{proximity}}$ ): 0.4**
- **Weight for Height ( $w_{\text{height}}$ ): 0.5**

These weights were selected to ensure that the fitness function appropriately rewards behaviours that enhance survival and adaptability in the simulated environment. The fitness functions design encourages the evolution of creatures that are not only mobile but also capable of navigating complex terrains and achieving higher altitudes on the mountain.

### Code Snippet for Fitness Calculation {-}

The following code snippet demonstrates how the fitness function is implemented in the simulation:

```python
def assess_fitness(self, cr, mountain_position):
    """
    Assess the creature's fitness based on its proximity to the mountain and its height
    """
    if cr.last_position is None:
        return

    pos = np.array(cr.last_position)
    mountain_base = np.array(mountain_position)
    distance_to_base = np.linalg.norm(pos[:2] - mountain_base[:2])

    # Calculate fitness components
    distance_travelled = np.linalg.norm(pos[:2] - np.array(cr.start_position)[:2])
    height = pos[2] if self.is_touching_mountain(pos) else 0

    # Weights
    weight_distance_travelled = 0.1
    weight_proximity = 0.4
    weight_height = 0.5

    # Calculate fitness
    fitness_distance_travelled = weight_distance_travelled * distance_travelled
    fitness_proximity = weight_proximity * (1 / (distance_to_base + 1))  # Inverse to minimize distance to base
    fitness_height = weight_height * height

    # Total fitness
    cr.fitness = fitness_distance_travelled + fitness_proximity + fitness_height
```

This function calculates the fitness of a creature by first determining its distance traveled from the starting position, its proximity to the mountain, and the height it has achieved whilst touching the mountain. These components are weighted and combined to form the overall fitness score, guiding the evolutionary process towards more capable and adaptive creatures.

## Results of Basic Experiments {-}

The results from the control experiment provided valuable insights into the effectiveness of the basic genetic algorithm setup. By tracking the mean fitness, maximum fitness, and the mean number of links in the creatures over 1000 iterations, we were able to observe the evolutionary progress and adaptiveness of the creatures.


### Mean Fitness Over Iterations {-}

In [ ]:
# Mean Fitness Over Iterations
plt.figure(figsize=(12, 6))
plt.plot(control_iterations, control_mean_fitness, label='Control Experiment', linewidth=2)
plt.xlabel('Iterations')
plt.ylabel('Mean Fitness')
plt.title('Mean Fitness Over Iterations')
plt.legend()
plt.grid(True)
plt.show()

The mean fitness shows a steady increase over iterations, indicating that the population is evolving towards higher fitness levels. This upward trend suggests that the genetic algorithm is effectively selecting and propagating fitter individuals.

**Insights:**

- Gradual Improvement: The steady rise in mean fitness reflects the gradual improvement of the population as beneficial traits become more prevalent.
- Algorithm Effectiveness: The increase in fitness levels confirms that the genetic algorithm is successfully identifying and promoting advantageous genetic combinations.


### Maximum Fitness Over Iterations {-}

In [ ]:
# Maximum Fitness Over Iterations
plt.figure(figsize=(12, 6))
plt.plot(control_iterations, control_max_fitness, label='Control Experiment', linewidth=2)
plt.xlabel('Iterations')
plt.ylabel('Maximum Fitness')
plt.title('Maximum Fitness Over Iterations')
plt.legend()
plt.grid(True)
plt.show()

The Maximum fitness plot highlights the highest fitness achieved in the population at each iteration. This metric is cruicial for understanding the potential of the genetic algorithm ot find highly fit individuals.

**Insights:**

- Peaks and Plateaus: The maximum fitness graph often shows peaks and plateaus, indicating periods of rapid improvement followed by stabilization as the algorithm converges on optimal solutions.
- Diversity and Exploration: The occasional sharp rises in maximum fitness suggest that the algorithm explores diverse genetic combinations, occasionally finding significantly better solutions.


### Mean Number of Links Over Iterations {-}

In [ ]:
# Mean Number of Links Over Iterations
plt.figure(figsize=(12, 6))
plt.plot(control_iterations, control_mean_links, label='Control Experiment', linewidth=2)
plt.xlabel('Iterations')
plt.ylabel('Mean Number of Links')
plt.title('Mean Number of Links Over Iterations')
plt.legend()
plt.grid(True)
plt.show()

The mean number of links indicate the complexity of the creatures. This plot helps understand if more complex creatures tend to evolve higher fitness levels.

**Insights**

- Complexity vs. Fitness: The relationship between the mean number of links and fitness could possibly indicate that more complex creatures are advantageous. An increase in the mean number of links alongside fitness improvements suggests that complexity contributes to better adaptation.

    
### Experiments with Encoding Scheme {-}
To further explore the evolutionary process, we conducted several experiments with different encoding schemes and parameter variations. These experiments aimed to understand how changes in encoding and mutation rates affect the evolution of the creatures.

**Experiment Variations**

Below is a table that summarizes the parameters and their encoding used in your experiments. Each experiment has specific parameters like population size, gene count, crossover rate, mutation rates, and any non-evolvable genes.

| Experiment Name                          | Population Size | Gene Count | Crossover Rate | Point Mutation Rate | Shrink Mutation Rate | Grow Mutation Rate | Non-Evolvable Genes                    |
|------------------------------------------|-----------------|------------|----------------|---------------------|----------------------|--------------------|----------------------------------------|
| Control Experiment                       | 20              | 3          | 0.9            | 0.1                 | 0.25                 | 0.1                | None                                   |
| Evolve Motor Controls                    | 20              | 3          | 0.9            | 0.1                 | 0.25                 | 0.1                | link-length, link-radius, link-shape   |
| Evolve Link Length and Motor Controls    | 20              | 3          | 0.9            | 0.1                 | 0.25                 | 0.1                | link-shape                             |
| High Population High Mutation            | 50              | 4          | 0.8            | 0.2                 | 0.35                 | 0.2                | None                                   |
| Low Population High Crossover            | 10              | 2          | 1.0            | 0.1                 | 0.25                 | 0.1                | link-radius                            |


## Results of Encoding Scheme Experiments {-}
The results of these experiments were compared to the control experiment to evaluate the impact of different encoding schemes on the evolution process. By analyzing the mean fitness, maximum fitness, and mean number of links over iterations, we can better understand how different genetic configurations influence the evolutionary outcomes.


### Mean Fitness Over Iterations {-}

In [ ]:
# Mean Fitness Over Interations (Analysis)
plt.figure(figsize=(12, 6))
plt.plot(control_iterations, control_mean_fitness, label='Control Experiment', linewidth=2)
for exp in encoding_experiments:
    plt.plot(encoding_metrics[exp][0], encoding_metrics[exp][1], label=exp.replace('_', ' ').title(), linewidth=2)
plt.xlabel('Iterations')
plt.ylabel('Mean Fitness')
plt.title('Mean Fitness Over Iterations')
plt.legend()
plt.grid(True)
plt.show()

The mean fitness plots for different experiments show varying rates of improvement. The experiment with high population and high mutation rates shows the fastest increase in mean fitness, indicating a more aggressive exploration of the fitness landscape. However, due to computational limitations, the system was unable to compute past the recorded iterations for this experiment. This highlights the trade-off between population size, mutation rates, and computational feasibility.


**Insights**

- Aggressive Exploration: The high population and high mutation rate experiment demonstrates a rapid increase in mean fitness, suggesting that a larger population combined with higher mutation rates can accelerate the discovery of beneficial traits.
- Computational Trade-offs: The inability to compute past certain iterations in the high population experiment underscores the balance between computational resources and the complexity of the genetic algorithm. While larger populations and higher mutation rates can drive faster evolution, they also demand more computational power.


### Maximum Fitness Over Iterations {-}

In [ ]:
# Maximum Fitness Over Iterations (Analysis)
plt.figure(figsize=(12, 6))
plt.plot(control_iterations, control_max_fitness, label='Control Experiment', linewidth=2)
for exp in encoding_experiments:
    plt.plot(encoding_metrics[exp][0], encoding_metrics[exp][2], label=exp.replace('_', ' ').title(), linewidth=2)
plt.xlabel('Iterations')
plt.ylabel('Maximum Fitness')
plt.title('Maximum Fitness Over Iterations')
plt.legend()
plt.grid(True)
plt.show()

The maximum fitness plots reveal that experiments with specific non-evolvable genes tend to reach higher fitness levels more quickly, suggesting that constraining certain traits can lead to more efficient optimization of others.

**Insights:**

- Efficient Optimization: Experiments with non-evolvable genes, such as those restricting link shapes or sizes, tend to reach higher maximum fitness levels more quickly. This suggests that limiting the evolution of certain traits can focus the genetic algorithm on optimizing other, more impactful traits.
- Rapid Peaks: The experiments with high mutation rates and large populations show sharp increases in maximum fitness at various points, indicating periods of rapid discovery of highly fit individuals. This is contrasted with the more gradual increases seen in experiments with more constraints.


### Mean Number of Links Over Iterations {-}

In [ ]:
# Mean Number of Links Over Iterations (Analysis)
plt.figure(figsize=(12, 6))
plt.plot(control_iterations, control_mean_links, label='Control Experiment', linewidth=2)
for exp in encoding_experiments:
    plt.plot(encoding_metrics[exp][0], encoding_metrics[exp][3], label=exp.replace('_', ' ').title(), linewidth=2)
plt.xlabel('Iterations')
plt.ylabel('Mean Number of Links')
plt.title('Mean Number of Links Over Iterations')
plt.legend()
plt.grid(True)
plt.show()

The complexity of the creatures, as indicated by the mean number of links, shows interesting trends. Experiments with high population sizes and mutation rates tend to produce more complex creatures, while those with non-evolvable genes maintain simpler structures.

**Insights:**
- Increased Complexity: Experiments with high population sizes and mutation rates result in creatures with more links, indicating greater complexity. This suggests larger populations and frequent mutations lead to more intricate structures.
- Simplicity and Efficiency: Experiments that constrain certain genes (non-evolvable) maintain simpler structures while achieving high fitness. This highlights that simplicity can be advantageous, and efficiency can be achieved without increasing structural complexity.
- Balancing Act: Comparing different encoding schemes shows a balance between complexity and efficiency. More complex creatures may have advantages, but simpler designs can also be effective depending on evolutionary constraints and objectives.


## Exceptional Criteria {-}
In addition to the standard experiments, we attempted an exceptional criterion by introducing sensory input when creatures touch the mountain. This additional feedback mechanism provided the creatures with enhanced motor controls, leading to interesting results. By implementing sensory input, the creatures were able to detect when they were in contact with the mountain and adjust their motor outputs accordingly. This added a layer of complexity and adaptability to the creatures’ behavior.

### Results of Exceptional Criteria {-}

The sensory input allowed creatures to adapt more effectively to the environment, showing significant improvements in both fitness and complexity. The ability to sense and react to environmental cues resulted in more adaptive and robust behaviors. For example, creatures that could detect the mountain would increase their motor output to climb it, leading to higher fitness scores. This experiment demonstrated that adding sensory input can significantly enhance the evolutionary process by promoting more sophisticated and capable behaviors.

To illustrate this, here is a code snippet that shows how sensory input was integrated into the motor control logic:

```python
def update_motors(self, cid, cr, target_position):
    """
    Update motors to control the creature's movements with sensory input
    """
    pos, orn = p.getBasePositionAndOrientation(cid, physicsClientId=self.physicsClientId)
    for jid in range(p.getNumJoints(cid, physicsClientId=self.physicsClientId)):
        m = cr.get_motors()[jid]
        boost = 1.0
        if self.is_touching_mountain(pos) or self.is_facing_mountain(pos, orn, target_position):
            boost = 1.5  # Increase motor output by 50% when touching the mountain
        p.setJointMotorControl2(cid, jid, controlMode=p.VELOCITY_CONTROL, targetVelocity=m.get_output() * boost, force=5, physicsClientId=self.physicsClientId)
```

This code shows how the creatures were programmed to boost their motor output upon sensing the mountain, leading to improved climbing abilities and overall fitness.

## Conclusion {-}
The experiments demonstrated the effectiveness of evolutionary computing in evolving creatures with varying fitness and complexity. The control experiment established a baseline, while the encoding scheme experiments revealed the impact of different genetic configurations on the evolutionary process. The exceptional criteria with sensory input highlighted the potential for more advanced adaptations. By adding sensory input, the creatures became more responsive to their environment, leading to significant improvements in their fitness and complexity.

Future work could explore more complex sensory inputs, multi-objective optimization, and real-world applications of evolved creatures in robotics and artificial intelligence. For instance, integrating sensors for light, sound, or other environmental factors could lead to even more sophisticated behaviors. Additionally, optimizing for multiple objectives, such as energy efficiency and speed, could create more well-rounded and practical solutions for real-world applications.


**Word Count: 1950**

## References {-}

Here are the references for the tools and libraries used in this project:

- NumPy Documentation (2024) NumPy. Available at: https://numpy.org/doc/ (Accessed: 8 July 2024).
- PyBullet Documentation (2024) PyBullet Physics Simulation. Available at: https://pybullet.org/ (Accessed: 8 July 2024).
- Matplotlib Documentation (2024) Matplotlib. Available at: https://matplotlib.org/stable/contents.html (Accessed: 8 July 2024).
- Seaborn Documentation (2024) Seaborn. Available at: https://seaborn.pydata.org/ (Accessed: 8 July 2024).
- Jupyter Notebook Documentation (2024) Jupyter Notebook. Available at: https://jupyter-notebook.readthedocs.io/ (Accessed: 8 July 2024).
- JSON Documentation (2024) JSON. Available at: https://www.json.org/json-en.html (Accessed: 8 July 2024).